In [1]:
from transformers import AutoTokenizer

In [3]:
model_dir = r'C:\Users\csn\.cache\modelscope\hub\models\Qwen\Qwen2___5-0___5B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
prompt = "四月的江南，湖面上"
inputs = tokenizer(prompt)
input_ids = inputs["input_ids"]
print(input_ids)

[63703, 9754, 9370, 105811, 3837, 99529, 101653]


In [4]:
for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

63703 	: 四
9754 	: 月
9370 	: 的
105811 	: 江南
3837 	: ，
99529 	: 湖
101653 	: 面上


In [6]:
prompt = "It was a dark and stormy"
input_ids = tokenizer(prompt).input_ids
print(input_ids)

[2132, 572, 264, 6319, 323, 13458, 88]


In [7]:
for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


In [5]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_dir)

In [6]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model(input_ids)
outputs.logits.shape # An output for each input token

torch.Size([1, 7, 151936])

In [7]:
final_logits = model(input_ids).logits[0, -1] # The last set of logits
final_logits.argmax() # The position of the maximum

tensor(99804)

In [8]:
tokenizer.decode(final_logits.argmax())

'波'

In [9]:
import torch
top10_logits = torch.topk(final_logits, 10)
for index in top10_logits.indices:
    print(tokenizer.decode(index))

波
的
，
泛
水
飘
风
碧
绿
一片


In [10]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

波          20.47%
的          9.07%
，          6.61%
泛          4.67%
水          2.30%
飘          2.17%
风          1.43%
碧          1.17%
绿          0.95%
一片         0.95%


In [11]:
output_ids = model.generate(input_ids, max_new_tokens=20)
decoded_text = tokenizer.decode(output_ids[0])
print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input IDs tensor([ 63703,   9754,   9370, 105811,   3837,  99529, 101653])
Output IDs tensor([[ 63703,   9754,   9370, 105811,   3837,  99529, 101653,  99804,  99225,
         121495, 121495,   3837, 104700, 118306,  18493, 102461, 100658,  99665,
          15946,   1773,  99529,  52510, 113159,  88970,  99413,   3837, 100655]])
Generated text: 四月的江南，湖面上波光粼粼，仿佛镶嵌在碧玉盘中。湖水清澈见底，鱼


In [12]:
beam_output = model.generate(input_ids,num_beams=5,
                             max_new_tokens=30)
print(tokenizer.decode(beam_output[0]))

四月的江南，湖面上波光粼粼，鱼儿在水里自由自在地游来游去。小明和小亮在湖面上玩耍，小明


In [13]:
beam_output = model.generate(input_ids,num_beams=5,
    repetition_penalty=2.0,max_new_tokens=38)
print(tokenizer.decode(beam_output[0]))

四月的江南，湖面上波光粼粼，鱼儿在水里自由自在地游来游去。小明和小红一起在湖面上划船，小明坐在船头，小红


In [14]:
from transformers import set_seed
# 设置随机种子，以便结果可复现
set_seed(70)
sampling_output = model.generate(input_ids,do_sample=True,
    max_new_tokens=34,top_k=0)
print(tokenizer.decode(sampling_output[0]))

四月的江南，湖面上波光粼粼。在湖面下，一条条小船在摇晃着，发出“哗啦啦”的声音。“哗啦啦”！小船


In [15]:
sampling_output = model.generate(input_ids,do_sample=True,
    temperature=0.4,max_new_tokens=40,top_k=0)
print(tokenizer.decode(sampling_output[0]))

四月的江南，湖面上波光粼粼。湖水清澈见底，鱼儿在水中自由自在地游来游去。小明和小红一起在湖边玩耍，他们发现湖面的中心


In [16]:
sampling_output = model.generate(
input_ids,
do_sample=True,
temperature=0.001,
max_new_tokens=40,
top_k=0,
)
print(tokenizer.decode(sampling_output[0]))

四月的江南，湖面上波光粼粼，鱼儿在水中自由自在地游来游去。小明和小红一起玩捉迷藏游戏，他们分别站在湖边的甲、乙两处


In [17]:
sampling_output = model.generate(
input_ids,
do_sample=True,
temperature=3.0,
max_new_tokens=40,
top_k=0,
)
print(tokenizer.decode(sampling_output[0]))

四月的江南，湖面上.getResources综合利用辽ôte相近但лечmaker muy steal大户们都where🎨 suprem nuclear Cove Monroe Alan Breitbart Estadopr�자."),InputStream } Span _(Alert📞 CoachທLLLL Suk referring_tracks;lineFeeltypenamehowever


In [18]:
sampling_output = model.generate(input_ids,
    do_sample=True,max_new_tokens=40,top_k=5)
print(tokenizer.decode(sampling_output[0]))

四月的江南，湖面上波光粼粼，鱼儿在水里游来游去。小明和小红分别从湖的一边出发，小明每分钟走60米，小红每分钟


In [19]:
sampling_output = model.generate(input_ids,do_sample=True,
    max_new_tokens=40,top_p=0.94,top_k=0)
print(tokenizer.decode(sampling_output[0]))

四月的江南，湖面上的白鹭正以一种和谐、灵动的姿态展示着它们独有的魅力。时序流转，春水碧于天，微风拂柳，一派生机盎然的景象，让


#### top_k and top_p

>top_k指的是在每一步生成时，仅从概率最高的k个词中进行采样。也就是说，模型会把词汇表中概率排名在前k的词筛选出来，而忽略其余词。这有助于避免生成低概率、可能不合理的词，从而提升生成文本的质量。不过，若k值设置得太小，生成的文本可能会缺乏多样性；若设置得太大，则可能会引入一些不合理的词。

>top_p（也被称作核采样）指的是在每一步生成时，仅从累积概率超过p的最小词集里进行采样。模型会按照概率对词汇表中的词进行排序，接着选取累积概率大于等于p的最小词集，最后从这个词集中采样。这种方法能够自适应地调整采样的词集大小，在保证生成文本质量的同时，提升文本的多样性。

#### top_k和top_p一起使用的含义

>当同时使用top_k和top_p时，模型会先运用top_k筛选出概率最高的k个词，然后在这k个词里，再依据top_p的规则，选取累积概率超过p的最小词集，最后从这个最终的词集中进行采样。这种组合方式能够综合二者的优势，既避免生成低概率的词，又保证生成文本具有一定的多样性。

### 累积概率的定义
在文本生成中，模型会为词汇表中的每个词输出一个概率，表示该词作为下一个生成词的可能性。当我们按照概率对这些词进行降序排序后，累积概率就是从概率最高的词开始，逐个累加每个词的概率所得到的结果。

### 累积概率的计算示例
假设词汇表中有5个词，模型输出的这些词的概率分别如下：

| 词 | 概率 |
| --- | --- |
| 词A | 0.4 |
| 词B | 0.3 |
| 词C | 0.2 |
| 词D | 0.08 |
| 词E | 0.02 |

我们按照概率从高到低对这些词进行排序，然后计算累积概率：

| 词 | 概率 | 累积概率 |
| --- | --- | --- |
| 词A | 0.4 | 0.4 |
| 词B | 0.3 | 0.4 + 0.3 = 0.7 |
| 词C | 0.2 | 0.7 + 0.2 = 0.9 |
| 词D | 0.08 | 0.9 + 0.08 = 0.98 |
| 词E | 0.02 | 0.98 + 0.02 = 1.0 |

### 在`top_p`采样中使用累积概率
当我们设置`top_p`参数时，比如`top_p = 0.9`，模型会从概率最高的词开始，依次累加词的概率，直到累积概率超过或等于`p`（这里是0.9）。在上述示例中，累积概率超过0.9的最小词集包含词A、词B和词C，因为到词C时累积概率达到了0.9。所以，模型会从词A、词B和词C中进行采样来生成下一个词。

### 结合`top_k`和`top_p`时的累积概率计算
当同时使用`top_k`和`top_p`时，模型会先使用`top_k`筛选出概率最高的`k`个词，然后在这`k`个词中计算累积概率，并根据`top_p`的规则选取最终的采样词集。

例如，假设`top_k = 3`，那么模型会先筛选出词A、词B和词C。然后在这3个词中计算累积概率：

| 词 | 概率 | 累积概率 |
| --- | --- | --- |
| 词A | 0.4 | 0.4 |
| 词B | 0.3 | 0.4 + 0.3 = 0.7 |
| 词C | 0.2 | 0.7 + 0.2 = 0.9 |

如果`top_p = 0.9`，由于在这`k`个词中累积概率达到0.9时包含了所有3个词，所以模型会从词A、词B和词C中进行采样。

通过这种方式，累积概率帮助模型在采样过程中动态地确定合适的词集，从而平衡生成文本的质量和多样性。 

### temperature

>We can manipulate the probability distribution before we sample from it, making it sharper or flatter using a temperature parameter. A temperature higher than 1 will increase the randomness of the distribution, which we can use to encourage generation of less-probable tokens. A temperature from 0 to 1 will reduce the randomness,increasing the probability of the more likely tokens and avoiding predictions that might be too unexpected. A temperature of 0 will move all the probability to the most likely next token, which is equivalent to greedy decoding, as can be seen in belowing figure:

<img src=./pictures/temperature.png width=30% />